In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

In [ ]:
CBWs         = [   2,   4,   6]
PBWs         = [  10,  16,  20]
CWNDs        = [  64, 128, 256]
TTIME        = 10
NUMEXP       = 20
TPTALGOs     = ['TCPreno','TCPcubic'] #,'UDP']
ATK_PARA     = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]
hostnames    = ['atkr'] + [f'h{i}' for i in range(8)]
iperf_header = ['timestamp','src_addr','src_port','dest_addr','dest_port','transferID','interval','transferred_bytes','bits_per_second']

indices      = [('CBW', CBWs), ('PBW', PBWs), ('CWND', CWNDs), ('TPTALGO', TPTALGOs), ('APC', ATK_PARA)]

basedir   = f'./NUMEXP-{NUMEXP}_TTIME-{TTIME}'

In [ ]:
dfall = []
for CBW in CBWs:
    for PBW in PBWs:
        for CWND in CWNDs: 
            for TPTALGO in TPTALGOs:
                for APC in ATK_PARA:
                    for HOST in hostnames:
                        dfi = pd.read_csv(f'{basedir}/CBW-{CBW}/PBW-{PBW}/CWND-{CWND}K/{TPTALGO}/{APC}conn/iperf_{HOST}.csv', names=iperf_header)
                        dfi['host']    = HOST
                        dfi['APC']     = APC
                        dfi['TPTALGO'] = TPTALGO
                        dfi['CWND']    = CWND
                        dfi['PBW']     = PBW
                        dfi['CBW']     = CBW
                        dfall.append(dfi)
                        
dfbigboi = pd.concat(dfall)
dfbigboi

In [ ]:
SCALAR_FIG_SZ = 5

for indeX in indices:
    for indeY in indices:
        if indices.index(indeX) == indices.index(indeY): continue
        AX_R_NAME, AX_R_VALS = indeX
        AX_C_NAME, AX_C_VALS = indeY
        AX_R_LEN , AX_C_LEN  = len(AX_R_VALS), len(AX_C_VALS)
        print(f'{AX_R_LEN} x {AX_C_LEN}\t', AX_R_NAME, '\tvs.  ', AX_C_NAME)
        
        fig, axs = plt.subplots(AX_R_LEN, AX_C_LEN, figsize=(AX_C_LEN*SCALAR_FIG_SZ, AX_R_LEN*SCALAR_FIG_SZ))
        
        maxY = 0
        for R_VAL in AX_R_VALS:
            df0 = dfbigboi[dfbigboi[AX_R_NAME] == R_VAL]
            for C_VAL in AX_C_VALS:
                df1 = df0[df0[AX_C_NAME] == C_VAL]
                axi = axs[AX_R_VALS.index(R_VAL)][AX_C_VALS.index(C_VAL)]
                
                net_data = [0] * len(range(TTIME))
                for HOST in hostnames:
                    df2 = df1[df1['host'] == HOST]
                    host_data = []
                    for t in range(TTIME):
                        tint = f'{t:.1f}-{t+1:.1f}'
                        df3 = df2[df2['interval'] == tint]
#                         print(HOST, df3.shape)
                        if HOST == 'atkr' and df3.shape[0] > 1:
                            df3 = df3[df3['transferID'] == -1]
#                         print(HOST, df3.shape)
                        host_total = df3.sum()['transferred_bytes'] // df3.shape[0]
#                         if(host_total != host_total):
#                             print(df3)
                        host_data.append(host_total)
                        net_data[t] += host_total
                    maxY = max(maxY, max(host_data), max(net_data))
#                     print(HOST, host_data)
                    axi.plot(range(TTIME), host_data, label=HOST)
                axi.plot(range(TTIME), net_data, label='network')
                axi.legend()
                axi.set_xlabel('time')
                axi.set_ylabel('transferred_bytes')
                axi.set_title(f'{AX_R_NAME} {R_VAL} vs. {AX_C_NAME} {C_VAL}')
                
        for R_VAL in AX_R_VALS:
            for C_VAL in AX_C_VALS:
                axi = axs[AX_R_VALS.index(R_VAL)][AX_C_VALS.index(C_VAL)]
                axi.set_ylim(bottom=0, top=maxY)
                
#         break
#     break

        fig.savefig(f'./VIZTIME/VIZTIME_{AX_R_NAME}-{AX_C_NAME}.png', facecolor="w")

In [20]:
SCALAR_FIG_SZ = 5

missing_data = 0
t_data = [t+0.5 for t in range(TTIME)]

maxY = 16140000 

for CBW in CBWs:
    df0 = dfbigboi[dfbigboi['CBW'] == CBW]
    for PBW in PBWs:
        df1 = df0[df0['PBW'] == PBW]
        for CWND in CWNDs: 
            df2 = df1[df1['CWND'] == CWND]
            for TPTALGO in TPTALGOs:
                df3 = df2[df2['TPTALGO'] == TPTALGO]
                for APC in ATK_PARA:
                    df4 = df3[df3['APC'] == APC]
                    print(CBW, PBW, CWND, TPTALGO, APC)
                    fig, axs = plt.subplots(1, 1, figsize=(SCALAR_FIG_SZ, SCALAR_FIG_SZ))
                    axs.set_ylim(bottom=-1, top=maxY)
                    net_data = [0] * len(t_data)
                    for HOST in hostnames:
                        host_data = []
                        df5 = df4[df4['host'] == HOST]
                        for t in range(TTIME):
                            tint = f'{t:.1f}-{t+1:.1f}'
                            df6 = df5[df5['interval'] == tint]
                            if df6.shape[0] > 1:
                                df6 = df6[df6['transferID'] == -1]
                            try:
                                host_data.append(df6['transferred_bytes'].values[0])
                            except:
                                host_data.append(-1)
                                missing_data += 1
                            net_data[t] += host_data[-1]
#                             maxY = max(maxY, host_data[-1], net_data[t])
                        axs.plot(t_data, host_data, label=HOST)
                    axs.plot(t_data, net_data, label='network')
                    axs.legend()
                    axs.set_xlabel('time')
                    axs.set_ylabel('transferred_bytes')
                    axs.set_title(f'CBW{CBW} PBW{PBW} {CWND}K {TPTALGO} APC{APC}')
                    fig.savefig(f'./VIZLITTLESHITS/CBW{CBW}-PBW{PBW}-{CWND}K-{TPTALGO}-APC{APC}.png', facecolor="w")
                    plt.close()
                    
print('missing_data:', missing_data)
print('maxY', maxY)

2 10 64 TCPreno 1
2 10 64 TCPreno 3
2 10 64 TCPreno 5
2 10 64 TCPreno 7
2 10 64 TCPreno 9
2 10 64 TCPreno 11
2 10 64 TCPreno 13
2 10 64 TCPreno 15
2 10 64 TCPreno 17
2 10 64 TCPreno 19
2 10 64 TCPreno 21
2 10 64 TCPcubic 1
2 10 64 TCPcubic 3
2 10 64 TCPcubic 5
2 10 64 TCPcubic 7
2 10 64 TCPcubic 9
2 10 64 TCPcubic 11
2 10 64 TCPcubic 13
2 10 64 TCPcubic 15
2 10 64 TCPcubic 17
2 10 64 TCPcubic 19
2 10 64 TCPcubic 21
2 10 128 TCPreno 1
2 10 128 TCPreno 3
2 10 128 TCPreno 5
2 10 128 TCPreno 7
2 10 128 TCPreno 9
2 10 128 TCPreno 11
2 10 128 TCPreno 13
2 10 128 TCPreno 15
2 10 128 TCPreno 17
2 10 128 TCPreno 19
2 10 128 TCPreno 21
2 10 128 TCPcubic 1
2 10 128 TCPcubic 3
2 10 128 TCPcubic 5
2 10 128 TCPcubic 7
2 10 128 TCPcubic 9
2 10 128 TCPcubic 11
2 10 128 TCPcubic 13
2 10 128 TCPcubic 15
2 10 128 TCPcubic 17
2 10 128 TCPcubic 19
2 10 128 TCPcubic 21
2 10 256 TCPreno 1
2 10 256 TCPreno 3
2 10 256 TCPreno 5
2 10 256 TCPreno 7
2 10 256 TCPreno 9
2 10 256 TCPreno 11
2 10 256 TCPreno 13
2 10 

In [ ]:
[t+0.5 for t in range(TTIME)]